In [1]:
import akshare as ak
import pandas as pd
import numpy as np
import fastparquet
from datetime import datetime, timedelta
import itertools
import requests
import time
import akshare as ak
from collections import deque
from IPython.display import display, clear_output
from concurrent.futures import ThreadPoolExecutor, as_completed
import concurrent.futures
import math
import glob
import os
import pyarrow.feather as feather

In [3]:
stock_df=ak.stock_zh_a_spot()
stock_df=stock_df[~stock_df['名称'].str.contains("ST")]
def keep_sh_sz(code):
    if code.startswith("sh") or code.startswith("sz"):
        return True
    return False
stock_df=stock_df[stock_df['代码'].apply(keep_sh_sz)]

#exclude the 30 创业板 and 68 科创板ß
stock_df=stock_df[~stock_df['代码'].str[2:4].isin(['30', '68'])]
stock_df['stockcode'] = stock_df['代码'].str[2:]

stock_df

,代码,名称,最新价,涨跌额,涨跌幅,买入,卖出,昨收,今开,最高,最低,成交量,成交额,时间戳,stockcode
264,sh600000,浦发银行,10.17,-0.07,-0.684,10.16,10.17,10.24,10.23,10.30,10.17,44651100.0,457102788.0,15:00:00,600000
265,sh600004,白云机场,9.11,-0.08,-0.871,9.11,9.12,9.19,9.18,9.21,9.08,14763552.0,135010636.0,15:00:01,600004
266,sh600006,东风股份,7.38,-0.31,-4.031,7.37,7.38,7.69,7.68,7.68,7.36,46614225.0,348576611.0,15:00:01,600006
267,sh600007,中国国贸,23.15,-0.13,-0.558,23.15,23.17,23.28,23.30,23.45,23.07,2074839.0,48204835.0,15:00:01,600007
268,sh600008,首创环保,3.01,-0.03,-0.987,3.01,3.02,3.04,3.04,3.05,3.01,84121321.0,254551468.0,15:00:01,600008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4020,sz003040,楚天龙,13.06,-0.89,-6.380,13.06,13.07,13.95,13.89,13.95,13.02,13479698.0,180943694.0,15:00:00,003040
4021,sz003041,真爱美家,25.40,-0.20,-0.781,25.40,25.45,25.60,25.62,27.50,25.35,3087752.0,80742581.0,15:00:00,003041
4022,sz003042,中农联合,14.92,-0.38,-2.484,14.92,14.93,15.30,15.21,15.25,14.81,4009751.0,60441227.0,15:00:00,003042
4023,sz003043,华亚智能,45.01,-1.00,-2.173,45.01,45.10,46.01,45.97,46.01,44.87,1431700.0,64972028.0,15:00:00,003043


In [11]:
stock_df.to_csv('/Users/lulu/Documents/STOCK/DATASET/STOCKCODE.csv', index=False)  # index=False to avoid writing row numbers

In [4]:
# Define the directory containing the Feather files
directory = '/Users/lulu/Documents/STOCK/DATASET/DAILY/'

# Get all Feather files in the directory
feather_files = [f for f in os.listdir(directory) if f.endswith('.feather')]

valid_files = []

# Extract valid dates from filenames
for file in feather_files:
    try:
        date_str = file.split('_')[-1].split('.')[0]
        file_date = datetime.strptime(date_str, "%Y-%m-%d")
        valid_files.append((file_date, file))
    except ValueError:
        continue  # Skip files with invalid date format

# Load the latest valid file
if valid_files:
    latest_file = max(valid_files, key=lambda x: x[0])[1]  # Get filename of latest date
    latest_filepath = os.path.join(directory, latest_file)
    
    # Load the latest daily data
    previous_daily_data = pd.read_feather(latest_filepath)
    previous_daily_data['date'] = pd.to_datetime(previous_daily_data['date'])
    
    print(f"Loaded data from: {latest_filepath}")
else:
    print("No valid Feather files found in the directory.")

Loaded data from: /Users/lulu/Documents/STOCK/DATASET/DAILY/daily_data_2025-02-27.feather


In [4]:
previous_daily_data['date'].unique()

<DatetimeArray>
['2024-12-11 00:00:00', '2021-01-04 00:00:00', '2021-01-05 00:00:00',
 '2021-01-06 00:00:00', '2021-01-07 00:00:00', '2021-01-08 00:00:00',
 '2021-01-11 00:00:00', '2021-01-12 00:00:00', '2021-01-13 00:00:00',
 '2021-01-14 00:00:00',
 ...
 '2025-02-12 00:00:00', '2025-02-13 00:00:00', '2025-02-14 00:00:00',
 '2025-02-17 00:00:00', '2025-02-18 00:00:00', '2025-02-19 00:00:00',
 '2025-02-20 00:00:00', '2025-02-21 00:00:00', '2025-02-24 00:00:00',
 '2025-02-25 00:00:00']
Length: 1002, dtype: datetime64[ns]

In [5]:
def fetch_daily_data(code, name, start_date, end_date, max_retries=4):
    """Fetch daily data for a single stock with retries and exponential backoff."""
    wait_time = 2  # Initial wait time

    for attempt in range(1, max_retries + 1):
        try:
            daily_data = ak.stock_zh_a_daily(symbol=code, start_date=start_date, end_date=end_date)
            if not daily_data.empty:
                daily_data['stockcode'] = code
                daily_data['stockname'] = name
                return daily_data[['stockcode', 'stockname', 'date', 'open', 'close', 'high', 'low', 'volume', 'amount', 'outstanding_share', 'turnover']]
        except (requests.exceptions.ConnectionError, ConnectionResetError) as e:
            print(f"Attempt {attempt} failed for {code} ({name}): {e}")
            time.sleep(wait_time)  # Exponential backoff
            wait_time *= 2  
        except Exception as e:
            print(f"Unexpected error for {code} ({name}): {e}")
            break  

    print(f"Failed to fetch {code} ({name}) after {max_retries} attempts.")
    return None

def get_daily_data(stock_df, start_date, end_date, max_workers=10):
    """Fetch daily data for all stocks concurrently with retries, displaying only the last 5 fetched stocks and count."""
    stock_daily = []
    error_codes = []
    counter = itertools.count(1)  # Thread-safe counter
    recent_stocks = deque(maxlen=5)  # Store only the last 5 fetched stocks

    total_stocks = len(stock_df)  # Total number of stocks to fetch

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_daily_data, row['代码'], row['名称'], start_date, end_date): (row['代码'], row['名称'])
                   for _, row in stock_df.iterrows()}

        for future in as_completed(futures):
            result = future.result()
            code, name = futures[future]
            fetched_count = next(counter)  # Increment fetched count

            if result is not None:
                stock_daily.append(result)
                recent_stocks.append(f"Stock {fetched_count}: {code} ({name})")

            else:
                error_codes.append(code)  

            # Clear previous outputs and print only the latest 5 stocks
            clear_output(wait=True)  # Clears the cell output in Jupyter
            print(f"Fetching stocks: {fetched_count}/{total_stocks}")  # Show fetching progress
            print("Latest 5 fetched stocks:")
            for stock in recent_stocks:
                print(stock)

    # Combine results
    combined_data = pd.concat(stock_daily, ignore_index=True) if stock_daily else pd.DataFrame()

    return combined_data, error_codes

In [10]:
# 25ms
# Step 1: Find the maximum date in your previous dataset
max_date = previous_daily_data['date'].max()

# Step 2: Calculate the next day's date (start_date for fetching new data)
next_day = pd.to_datetime(max_date) + timedelta(days=1)

# Step 3: Get today's date (end_date for fetching new data)
today_date = datetime.today().strftime('%Y-%m-%d')

# Step 4: Set the start and end dates for fetching the new data
start_date = next_day.strftime('%Y-%m-%d')
end_date = today_date

# Step 5: Call the `get_daily_data` function to fetch the latest data
daily_data, error_codes = get_daily_data(stock_df, start_date, end_date)

Fetching stocks: 32/3079
Latest 5 fetched stocks:
Stock 28: sh600036 (招商银行)
Stock 29: sh600039 (四川路桥)
Stock 30: sh600037 (歌华有线)
Stock 31: sh600038 (中直股份)
Stock 32: sh600031 (三一重工)
Failed to fetch sh600114 (东睦股份) after 4 attempts.
Failed to fetch sh600168 (武汉控股) after 4 attempts.
Failed to fetch sh600225 (退市卓朗) after 4 attempts.
Attempt 1 failed for sh600373 (中文传媒): ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Attempt 1 failed for sh600377 (宁沪高速): ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Attempt 1 failed for sh600378 (昊华科技): ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Attempt 2 failed for sh600373 (中文传媒): ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Attempt 1 failed for sh600421 (华嵘控股): ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Attempt 1 failed for sh600418 (江淮汽车): ('Connection aborted.', ConnectionResetError(54, 'Co

KeyboardInterrupt: 

Attempt 1 failed for sh600717 (天津港): ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Attempt 1 failed for sh600726 (华电能源): ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))Attempt 1 failed for sh600725 (云维股份): ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Attempt 1 failed for sh600743 (华远地产): ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Attempt 1 failed for sh600784 (鲁银投资): ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Attempt 1 failed for sh600841 (动力新科): ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Attempt 1 failed for sh600850 (电科数字): ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Attempt 1 failed for sh600860 (京城股份): ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Attempt 1 failed for sh600866 (星湖科技): ('Connection aborted.', ConnectionRe

In [7]:
error_codes

['sh600114',
 'sh600168',
 'sh600225',
 'sh600539',
 'sh600837',
 'sh603626',
 'sh603617',
 'sh603618',
 'sh603613',
 'sh603609',
 'sh603628',
 'sh603616',
 'sh603615',
 'sz002725']

In [8]:
# Check if error_codes is not empty
if error_codes:
    # Filter stock_df for codes in error_codes
    error_daily_df = stock_df[stock_df['代码'].isin(error_codes)]
    
    # Fetch daily data for the stocks in error_codes
    daily_data_error, error_daily_2 = get_daily_data(error_daily_df, start_date, end_date)
else:
    print("No error codes found, skipping data fetch.")

daily_data_latest = pd.concat([daily_data, daily_data_error], ignore_index=True)
daily_data_latest

Fetching stocks: 14/14
Latest 5 fetched stocks:
Stock 4: sh603609 (禾丰股份)
Stock 5: sh603617 (君禾股份)
Stock 6: sh603628 (清源股份)
Stock 7: sh603626 (科森科技)
Stock 8: sh603618 (杭电股份)


,stockcode,stockname,date,open,close,high,low,volume,amount,outstanding_share,turnover
0,sh600010,包钢股份,2025-02-28,1.85,1.81,1.87,1.81,391823476.0,7.203411e+08,3.149712e+10,0.012440
1,sh600012,皖通高速,2025-02-28,17.73,17.71,17.87,17.43,7966514.0,1.408911e+08,1.165600e+09,0.006835
2,sh600008,首创环保,2025-02-28,3.04,3.01,3.05,3.01,84121321.0,2.545515e+08,7.340591e+09,0.011460
3,sh600006,东风股份,2025-02-28,7.68,7.38,7.68,7.36,46614225.0,3.485766e+08,2.000000e+09,0.023307
4,sh600007,中国国贸,2025-02-28,23.30,23.15,23.45,23.07,2074839.0,4.820484e+07,1.007283e+09,0.002060
...,...,...,...,...,...,...,...,...,...,...,...
3068,sh603609,禾丰股份,2025-02-28,7.53,7.50,7.62,7.48,7259348.0,5.473190e+07,9.194344e+08,0.007895
3069,sh603617,君禾股份,2025-02-28,8.11,7.62,8.12,7.61,29842124.0,2.331103e+08,3.800813e+08,0.078515
3070,sh603628,清源股份,2025-02-28,12.88,12.86,13.09,12.81,9034700.0,1.169583e+08,2.721684e+08,0.033195
3071,sh603626,科森科技,2025-02-28,10.43,10.58,11.28,10.16,156643805.0,1.680762e+09,5.548797e+08,0.282302


In [9]:
error_daily_2

['sh600837', 'sh600168', 'sh600539', 'sh600114', 'sh600225', 'sz002725']

In [11]:
# Optionally, append the new data to the existing dataset
if not daily_data_latest.empty:
    all_daily_data = pd.concat([previous_daily_data, daily_data_latest], ignore_index=True)
    all_daily_data['date']=pd.to_datetime(all_daily_data['date']).astype(str)
    print("Combined data with new daily stock data.")
else:
    print("No new data fetched.")

# Step 6: Save the combined data to a Parquet file with the latest date in the filename
if not daily_data_latest.empty:
    # Convert the 'date' column to string format if required for fastparquet
    all_daily_data['date'] = pd.to_datetime(all_daily_data['date']).astype(str)
    
    # Use f-string for dynamically inserting the end_date
    filename = f'/Users/lulu/Documents/STOCK/DATASET/DAILY/daily_data_{end_date}.feather'
    #filename = f"/Users/lulu/Documents/STOCK/combined_daily_data_{end_date}.feather" 
    
    # Save to Parquet using fastparquet engine
    feather.write_feather(all_daily_data, filename)
    #daily_data_latest.to_parquet(filename, engine='fastparquet')
    print(f"Saved the combined data to: {filename}")

Combined data with new daily stock data.
Saved the combined data to: /Users/lulu/Documents/STOCK/DATASET/DAILY/daily_data_2025-02-27.feather
